In [1]:
#pyCUDA 및 openCV를 이용한 이미지에 노이즈를 추가하는 코드

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import cv2 as cv
import numpy as np

import time

from numba import jit

In [3]:
img = cv.imread("C:/Users/dev/Desktop/LiveHDRI/OpenCV_test/imageset/landscape.jpg")

gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray_img = gray_img.astype(np.float32)
print(gray_img.shape)
print(gray_img.dtype)

(551, 980)
float32


In [4]:
#testimg = gray_img.astype(np.uint8)
#cv.imshow("landscape",testimg)
#cv.waitKey(0)

In [5]:
noise = np.random.randn(gray_img.shape[0],gray_img.shape[1])
noise = noise.astype(np.float32)
print(noise[1,1].dtype)
print(noise.shape)

float32
(551, 980)


In [6]:
result_img = np.zeros((gray_img.shape[0],gray_img.shape[1]),dtype='float32')
result2_img = np.zeros((gray_img.shape[0],gray_img.shape[1]),dtype='float32')
print(result_img.shape)

(551, 980)


In [7]:
print(gray_img.nbytes)

2159920


In [8]:
start_time = time.time()

img_gpu = cuda.mem_alloc(gray_img.nbytes)
cuda.memcpy_htod(img_gpu,gray_img)

img_noise_gpu = cuda.mem_alloc(noise.nbytes)
cuda.memcpy_htod(img_noise_gpu,noise)

img_result_gpu = cuda.mem_alloc(result_img.nbytes)
cuda.memcpy_htod(img_result_gpu,result_img)


In [9]:
kernel = SourceModule("""
    __global__ void addnoise(float* input, float* output, float* noise){

        int block_id =
            blockIdx.x +
            blockIdx.y * gridDim.x;

        int block_offset =
            block_id *
            blockDim.x * blockDim.y * blockDim.z;

        int thread_offset =
            threadIdx.x +
            threadIdx.y * blockDim.x;

        int id = block_offset + thread_offset;

        int limit_id = 539980;
        int blocksize = 1024;
        //output[id] = input[id] + 10*noise[id];

        if ( id < 539980) {
            output[id] = input[id] + 10*noise[id];
        }
    }
""")

In [10]:
threadsPerBlock = (32,32,1)
dx, mx = divmod(gray_img.shape[0], threadsPerBlock[0])
dy, my = divmod(gray_img.shape[1], threadsPerBlock[1])

blocksPerGrid = ((dx + (mx>0)), (dy + (my>0)))
#blocksPerGrid = (200, 350)
print(blocksPerGrid)

blocksPerGrid=(528,1)
func = kernel.get_function("addnoise")

func(img_gpu,img_result_gpu,img_noise_gpu, block=threadsPerBlock, grid=blocksPerGrid)

(18, 31)


In [11]:
cuda.memcpy_dtoh(result_img,img_result_gpu)
print("%s seconds in GPU" %(time.time()-start_time))

0.06250452995300293 seconds in GPU


In [12]:
start_time2 = time.time()
for i in range(551):
    for j in range(980):
        result2_img[i,j] = gray_img[i,j] + noise[i,j]

print("%s seconds in CPU" %(time.time()-start_time2))

0.325775146484375 seconds in CPU


In [13]:
# start_time3 = time.time()
# @jit(nopython=True)
# def imgloop():
#     for i in range(551):
#         for j in range(980):
#             result2_img[i,j] = gray_img[i,j] + noise[i,j]
# imgloop()
# print("%s seconds in CPU with numba" %(time.time()-start_time3))


print(result_img[550,979])
result_img = result_img.astype(np.uint8)
print(result_img.shape)
print(result_img)
cv.imshow("gpu_result",result_img)
cv.waitKey(0)

44.243748
(551, 980)
[[ 65  78  66 ...  73  74  71]
 [ 63  70  72 ...  84  50  79]
 [ 69  77  79 ...  69  75  80]
 ...
 [ 88  49  34 ...  14   2  25]
 [104  78  22 ...  29  25  32]
 [ 96  53  85 ...  56  20  44]]


-1